In [1]:
from collections import namedtuple
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path

In [3]:
import cv2

In [4]:
from skimage.transform import resize

ModuleNotFoundError: No module named 'skimage'

In [ ]:
from skimage.io import imread, imsave

In [61]:
with open('eigen_split.txt', 'r') as f:
    eigen_split = f.readlines()
    eigen_split = [e.rstrip() + "_sync" for e in eigen_split if e is not "\n"]
    eigen_split = np.array(eigen_split)

In [62]:
len(np.unique(eigen_split)), len(eigen_split)

(61, 61)

In [63]:
eigen_split_index = np.random.choice( np.arange(len(eigen_split)).astype(int), replace=False, size=len(eigen_split))

In [64]:
eigen_split_train = eigen_split[eigen_split_index[:len(eigen_split)//2]]
eigen_split_val = eigen_split[eigen_split_index[len(eigen_split)//2:]]

In [65]:
eigen_split_index

array([38, 32, 44, 53, 12, 15, 10,  5, 59, 49, 43,  3, 24, 23, 34, 33, 11,
       35, 57, 17, 45, 21, 25, 14, 47, 22, 19,  2,  6, 41, 46, 50, 31, 16,
       40,  0, 54,  4, 13, 56, 27, 39, 37, 28,  9, 51,  7, 42, 29,  8, 60,
       48, 58, 20,  1, 18, 26, 30, 52, 55, 36])

In [66]:
len(np.unique(eigen_split_index))

61

In [69]:
set(eigen_split_train) & set(eigen_split_val)

set()

In [70]:
FORCE = False

In [71]:
USE_CV = True

def load_img(path):
    if USE_CV:
        return cv2.imread(path)
    else:
        return imread(path)

def save_img(path, img):
    if USE_CV:
        return cv2.imwrite(path, img)
    else:
        return imsave(path, img)
    
def resize_img(img, size):
    if USE_CV:
        return cv2.resize(img, size)
    else:
        return resize(
            img,
            size,
            mode="reflect"
            #anti_aliasing=True
        )

In [72]:
if USE_CV:
    size = (640, 192)
    org_size = (1242, 375)
else:
    size = (192, 640)
    org_size = (375, 1242)
dscale = np.array(size)/np.array(org_size)
dscale = np.concatenate((dscale, dscale))

In [73]:
dscale

array([0.51529791, 0.512     , 0.51529791, 0.512     ])

In [74]:
roots = Path('.').iterdir()
roots=list(roots)
roots = [root for root in roots if root.is_dir() and '2011_' in root.name]
roots

[PosixPath('2011_09_29'),
 PosixPath('2011_09_30'),
 PosixPath('2011_09_26'),
 PosixPath('2011_09_28'),
 PosixPath('2011_10_03')]

size = (188, 620)
#size = (620, 188)
for root in roots:
    folders = root.glob("*_sync")
    for folder in folders:
        img02r = folder /'image_02_{}_{}'.format(*size)/'data'
        img03r = folder /'image_03_{}_{}'.format(*size)/'data'

        for which in (img02r, img03r):
            !rm -r {str(which)}

In [75]:
from tqdm import tqdm

In [76]:
timgs0, timgs1, timgs2, paras = [], [], [], []


readimgs = []
saveimgs = []

for root in roots:
    folders = root.glob("*_sync")
    for folder in folders:
        #if folder.name not in eigen_split: continue
            
        img02 = folder /'image_02'/'data'
        img03 = folder /'image_03'/'data'
        img02r = folder /'image_02_{}_{}'.format(*size)/'data'
        img03r = folder /'image_03_{}_{}'.format(*size)/'data'
        
        img02r.mkdir(parents=True, exist_ok=True)
        img03r.mkdir(parents=True, exist_ok=True)
        for which, which_r in ((img02, img02r), (img03, img03r)):
            imgs = sorted([img for img in which.iterdir()])
            readimgs += imgs
            saveimgs += [which_r/img.name for img in imgs]

total = len(readimgs); total                

95869

In [77]:
from multiprocessing import Process, Lock

def f(readimgs, saveimgs):
    pbar = tqdm(total=len(readimgs))
    for img, saveimg in zip(readimgs, saveimgs):
        if not saveimg.exists() or FORCE:
            resized = resize_img( load_img(str(img)), size)
            save_img(str(saveimg), resized)        
        pbar.update(1)
    pbar.close

In [78]:
ncores = 6
space = int( total / ncores ) + 1 
ps = []
space

15979

In [79]:
for num in range(ncores):
    p = Process(target=f,
                args=(readimgs[space*num: space*(num+1)],
                      saveimgs[space*num: space*(num+1)],
                      )
               )
    ps.append(p)
    
for p in ps: p.start()

 79%|███████▉  | 12627/15979 [04:33<00:05, 609.36it/s]

In [81]:
for p in ps: p.terminate()

In [ ]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
pbar = tqdm(total=total)
for root in roots:
    folders = root.glob("*_sync")
    for folder in folders:
        #if folder.name not in eigen_split: continue
            
        img02 = folder /'image_02'/'data'
        img03 = folder /'image_03'/'data'
        img02r = folder /'image_02_{}_{}'.format(*size)/'data'
        img03r = folder /'image_03_{}_{}'.format(*size)/'data'
        
        img02r.mkdir(parents=True, exist_ok=True)
        img03r.mkdir(parents=True, exist_ok=True)
        for which, which_r in ((img02, img02r), (img03, img03r)):
            imgs = sorted([img for img in which.iterdir()])
            
            for img in imgs:
                img_r = which_r/img.name
                if not img_r.exists() or FORCE:
                    resized = resize_img( load_img(str(img)), size)
                    save_img(str(img_r), resized)
                pbar.update(1)
pbar.close()

In [82]:
# CAMPARA = namedtuple('campara',['fx','fy','cx','cy'])
def get_campara(date):
    cam = date/'calib_cam_to_cam.txt'
    camret = {}
    with cam.open('r') as f:
        for l in f:
            kv = l.rstrip().split(':')
            k, v = kv[0], kv[1]
            v= v.split(' ')[1:]
            camret[k] = v
#     cam2para_2 = np.array(tuple(float(e) for e in camret['P_rect_02'])) #.reshape(3,4)
#     cam2para_3 = np.array(tuple(float(e) for e in camret['P_rect_03']))#.reshape(3,4)
    return CAMPARA(fx = cam2para[0,0], fy = cam2para[1,1], cx = cam2para[0,2], cy = cam2para[1,2])
#     return cam2para_2, cam2para_3

In [83]:
def gene_tri(imgs):
    return imgs[0:-2], imgs[1:-1], imgs[2:]

In [84]:
for name, split in zip(("training" , "validation"), (eigen_split_train, eigen_split_val)):
    timgs0, timgs1, timgs2, paras = [], [], [], []
    for root in roots:
        para2, para3 = get_campara(root)
        inv_2, inv_3
        folders = root.glob("*_sync")
        for folder in folders:
            if folder.name not in split: continue

            img02r = folder /'image_02_{}_{}'.format(*size)/'data'
            img03r = folder /'image_03_{}_{}'.format(*size)/'data'

            for which, para in zip(((img02r, img03r),(para2,para3))):
                imgs = sorted([str(img) for img in which.iterdir()])
                imgs0, imgs1, imgs2 = gene_tri(imgs)
                timgs0 += imgs0
                timgs1 += imgs1
                timgs2 += imgs2
                paras += [para]*len(imgs0)
                
#     paras = np.stack(paras, axis=0)
    paras = tuple(zip(*paras))
    
    train = pd.DataFrame(dict(t0=timgs0, t1=timgs1,t2=timgs2,
                          fx=paras[0],fy=paras[1],
                          cx=paras[2],cy=paras[3])
                    )

    train[['fx','fy','cx','cy']]=train[['fx','fy','cx','cy']]*dscale
    
    
    if USE_CV:
        train.to_csv('{0}_{2}_{1}.csv'.format(name, *size), index=False)
    else:
        train.to_csv('{}_{}_{}.csv'.format(name, *size), index=False)

# Remove Static Frame

In [85]:
if USE_CV:
    train = pd.read_csv('training_{1}_{0}.csv'.format(*size))
    val = pd.read_csv('validation_{1}_{0}.csv'.format(*size))
else:
    train = pd.read_csv('training_{}_{}.csv'.format(*size))
    val = pd.read_csv('validation_{}_{}.csv'.format(*size))    

In [86]:
train.loc[0, 't0']

'2011_09_29/2011_09_29_drive_0004_sync/image_02_640_192/data/0000000000.png'

In [87]:
staticf = Path('static_frames.txt').open()

In [88]:
staticf = staticf.readlines()

In [89]:
staticf[0].rstrip().split(' ') 

['2011_09_26', '2011_09_26_drive_0009_sync', '0000000386']

In [90]:
staticf_folder = [ "/".join( line.rstrip().split(' ')[:-1] ) for line in staticf ]
staticf_file = [ line.rstrip().split(' ')[-1]  for line in staticf ]

In [91]:
img2, img3 = "image_02_{0}_{1}".format(*size), "image_03_{0}_{1}".format(*size)

In [92]:
count = 0
for f, file in zip(staticf_folder, staticf_file):
    for img in img2, img3:
        path =  Path(f + "/" + img + "/data/" + file +'.png')
        if path.exists(): 
            path.unlink()
            count += 1
            
count

16071

In [93]:
for sheet in (train, val):
    drops = [] 
    for i in range(len(sheet)):
        for c in ("t0","t1","t2"):
            if not Path(sheet.loc[i, c]).exists():
                drops.append(i)
    sheet.drop(drops, axis=0, inplace=True)

In [94]:
if USE_CV:
    val.to_csv('validation_{1}_{0}_pre.csv'.format(*size), index=False)
    train.to_csv('training_{1}_{0}_pre.csv'.format(*size), index=False);
else:
    val.to_csv('validation_{}_{}_pre.csv'.format(*size), index=False)
    train.to_csv('training_{}_{}_pre.csv'.format(*size), index=False);

In [95]:
val.index

Int64Index([   17,    18,    19,    20,    21,    22,    23,    24,    25,
               26,
            ...
            36754, 36755, 36756, 36757, 36758, 36759, 36760, 36761, 36762,
            36763],
           dtype='int64', length=33744)

In [96]:
train.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            47690, 47691, 47692, 47693, 47694, 47695, 47696, 47697, 47698,
            47699],
           dtype='int64', length=44740)